In [ ]:
from google.colab import userdata
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import wandb
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainingArguments, SentenceTransformerTrainer, losses
from sentence_transformers.evaluation import SentenceEvaluator
from datasets import load_dataset

In [ ]:
folder = '/content/drive/MyDrive/conserta-avioes'

In [ ]:
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

# Datasets e Funções

## Dataset de Treino

In [ ]:
dataset_treino = load_dataset(folder,data_files="dataset_treino_pronto.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_treino = dataset_treino['train'].shuffle()
dataset_treino.features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': Value('int64')}

## Dataset de Teste

In [ ]:
dataset_teste = pd.read_csv(folder+"/dataset_teste.csv")

In [ ]:
corpus_teste = dataset_teste.loc[dataset_teste['Tipo']=='orig','Frase'].to_list()
corpus_teste[:10]

['RIGHT ENGINE #4 AIR BAFFLE IS CRACKED',
 'RIGHT ENGINE FORWARD ALTERNATOR ATTACH BOLT LOOSE.',
 'AFTER LANDING, A/C IDLE @ 970 RPM.',
 'TOP FRONT RIGHT BAFFLE, BAFFLE SEAL RIVET PULLED THROUGH.',
 'ROUGH RUNNING ENGINE ON START. ENGINE RAN SMOOTHER AS IT WAR',
 'CYLINDER HEAD TEMPERATURE NEEDLE BOUNCES & HAD ENGINE RUN ROUGH MOMENTARILY.',
 'LACING CORD LOOSE ON SCAT TUBING + IGNITION LEAD TO FRAME, RIGHT SI',
 'RIGHT SIDE BACK BAFFLE IS CRACKED & BRACKET RIVETS BROKEN.',
 'SPARK PLUG BAFFLE PLUG IS WORN.',
 '4TH STAGE NOZZLE HAS SEVERAL CRACKS IN IT.']

In [ ]:
queries_teste = dataset_teste.loc[dataset_teste['Tipo']=='var','Frase'].apply(lambda x: x.upper().strip()).to_list()
queries_teste[:10]

['CRACKED AIR BAFFLE DETECTED ON THE RIGHT ENGINE, #4.',
 'FOUND A CRACK IN THE LEFT ENGINE’S #3 AIR BAFFLE DURING INSPECTION.',
 'THE AIR BAFFLE ON ENGINE #2 (RIGHT SIDE) SHOWS A CRACK.',
 'LEFT ENGINE #4 AIR BAFFLE IS CRACKED.',
 'THE FORWARD ALTERNATOR ATTACH BOLT ON THE RIGHT ENGINE WAS FOUND LOOSE.',
 'FOUND THE ALTERNATOR FORWARD MOUNTING BOLT LOOSE ON THE LEFT ENGINE.',
 'ON ENGINE #2 THE FORWARD BOLT THAT SECURES THE ALTERNATOR IS LOOSE.',
 'NO. 1 ENGINE FORWARD ALTERNATOR ATTACH BOLT IS LOOSE.',
 'AFTER LANDING, A/C IDLE AT 970 RPM.',
 'UPON TOUCHDOWN THE AIRPLANE REMAINED IDLING AROUND 960 RPM.']

## Dataset de Validação

### Treino

In [ ]:
dataset_val = load_dataset(folder,data_files="dataset_validacao_pronto.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_val = dataset_val['train'].shuffle()
dataset_val.features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': Value('int64')}

### Recall @ K

In [ ]:
dataset_val_bruto = pd.read_csv(folder+"/dataset_validacao.csv")

In [ ]:
corpus_val = dataset_val_bruto.loc[dataset_val_bruto['Tipo']=='orig','Frase'].to_list()
corpus_val[:10]

['HOLDING SHORT OF RUNWAY, AFTER COMPLETION OF RUN UP, ENGINE',
 'IN FLIGHT, ENGINE BECAME ROUGH. DISCOVERED RIGHT MAGNETOS INOPERABLE. E',
 'LAST DAILY SHUT DOWN IDLE OVER RIDE SPEED CHECK - ENGINE RPM',
 'OIL FOUND ON ENTIRE RIGHT SIDE OF FUSELAGE.',
 'BAFFLE SEALS UNDER BOTH PROP GOVERNORS ARE LOOSE.',
 'RIGHT ENGINE, FORWARD I/B BAFFLE HAS SMALL CRACK.',
 'FELT ON #1 & 3 BAFFLE COMING LOOSE.',
 'FORWARD PUSH ROD SEAL LEAKING @ ENGINE ON #3 CYLINDER.',
 'BAFFLE SEAL HAS HOLE IN IT.',
 'CYLINDER #3 AFT BAFFLE MOUNT LOOSE.']

In [ ]:
queries_val = dataset_val_bruto.loc[dataset_val_bruto['Tipo']=='var','Frase'].apply(lambda x: x.upper().strip()).to_list()
queries_val[:10]

['HOLDING SHORT OF THE RUNWAY AFTER COMPLETION OF THE RUN-UP, THE ENGINE GAUGES WERE MONITORED.',
 'STOPPED SHORT OF THE RUNWAY ONCE THE RUN-UP WAS DONE AND WE KEPT AN EYE ON THE ENGINE.',
 'AFTER FINISHING THE RUN-UP AND HOLDING SHORT OF THE RUNWAY, THE ENGINE REMAINED UNDER OBSERVATION.',
 'RUN-UP COMPLETE, HOLDING SHORT OF RUNWAY — ENGINE BEING CHECKED.',
 'DURING FLIGHT THE ENGINE BEGAN RUNNING ROUGH; INSPECTION REVEALED THE RIGHT MAGNETOS WERE INOPERATIVE.',
 'THE ENGINE GOT ROUGH IN THE AIR AND WE FOUND THE LEFT MAGNETO HAD FAILED.',
 'IN-FLIGHT ENGINE ROUGHNESS NOTED — DISCOVERED THE #2 MAGNETO NOT WORKING.',
 'ENGINE STARTED ROUGHING OUT WHILE AIRBORNE; THE RIGHT MAG WAS FOUND INOPERATIVE.',
 'DURING THE LAST DAILY SHUTDOWN THE IDLE-OVERRIDE SPEED WAS CHECKED AND ENGINE RPM RECORDED.',
 'AT THE LAST DAILY SHUT-DOWN WE RAN AN IDLE OVERRIDE SPEED CHECK AND NOTED THE ENGINE RPM.']

## Recall @ K

In [ ]:
def recall_at_k(model,k,corpus,queries,dist='cos'):
    if len(queries) != 4*len(corpus):
      print("Tamanho errado para as listas de corpus e queries.")
      return 0
    gabarito = np.repeat(np.identity(len(corpus)),4,axis=0)
    with torch.no_grad():
            corpus_emb = model.encode(corpus, convert_to_tensor=True, show_progress_bar=False)
            query_emb = model.encode(queries, convert_to_tensor=True, show_progress_bar=False)
            sims = model.similarity(query_emb,corpus_emb)
    sims_np = sims.cpu().numpy()
    ranks = np.argsort(np.argsort(-sims_np, axis=1),axis=1)
    topk = ranks < k
    encontradas = np.logical_and(topk,gabarito).astype(int).sum()
    return encontradas/len(queries)

In [ ]:
class RecallAtKEvaluator(SentenceEvaluator):
    def __call__(
        self, model: SentenceTransformer, output_path: str | None = None, epoch: int = -1, steps: int = -1
    ) -> float | dict[str, float]:
        recall_1 = recall_at_k(model,1,corpus_val,queries_val)
        recall_3 = recall_at_k(model,3,corpus_val,queries_val)
        recall_7 = recall_at_k(model,7,corpus_val,queries_val)
        return {"recall@1":recall_1,"recall@3":recall_3,"recall@7":recall_7}

# Treinamento

In [ ]:
modelos = [
    "all-MiniLM-L6-v2",
    "all-MiniLM-L12-v2",
    "multi-qa-mpnet-base-dot-v1",
    "multi-qa-mpnet-base-cos-v1",
    "multi-qa-distilbert-cos-v1",
    "multi-qa-MiniLM-L6-cos-v1",
]

learning_rates = [5e-06,5e-05]

In [ ]:
resultados = pd.DataFrame(columns=['Modelo Base','Learning Rate','Recall@1','Recall@3','Recall@7'])
resultados['Modelo Base'] = modelos+modelos
resultados['Learning Rate'] = [5e-06]*6+[5e-05]*6
resultados

,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
0,all-MiniLM-L6-v2,0.000005,NaN,NaN,NaN
1,all-MiniLM-L12-v2,0.000005,NaN,NaN,NaN
2,multi-qa-mpnet-base-dot-v1,0.000005,NaN,NaN,NaN
3,multi-qa-mpnet-base-cos-v1,0.000005,NaN,NaN,NaN
4,multi-qa-distilbert-cos-v1,0.000005,NaN,NaN,NaN
5,multi-qa-MiniLM-L6-cos-v1,0.000005,NaN,NaN,NaN
6,all-MiniLM-L6-v2,0.000050,NaN,NaN,NaN
7,all-MiniLM-L12-v2,0.000050,NaN,NaN,NaN
8,multi-qa-mpnet-base-dot-v1,0.000050,NaN,NaN,NaN
9,multi-qa-mpnet-base-cos-v1,0.000050,NaN,NaN,NaN


In [ ]:
modelo = modelos[2]
learning_rate = learning_rates[0]
resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate)]

,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
2,multi-qa-mpnet-base-dot-v1,0.000005,NaN,NaN,NaN


In [ ]:
for i,modelo in enumerate(modelos):
  for learning_rate in learning_rates:
    print(f"Treinando modelo '{modelo}' (modelo {i+1} de 6) com learning rate de {learning_rate}:")
    nome = f"{modelo}_{learning_rate}"
    epocas = 4

    run = wandb.init(project="conserta-avioes_fine-tuning_01",name=nome)

    model = SentenceTransformer(modelo)
    train_loss = losses.CosineSimilarityLoss(model=model)

    args = SentenceTransformerTrainingArguments(
      output_dir=nome,
      num_train_epochs=epocas,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      eval_strategy="steps",
      eval_steps=100,
      save_strategy="steps",
      save_steps=100,
      save_total_limit=2,
      logging_steps=100,
      load_best_model_at_end=True,
      metric_for_best_model="recall@1",
      greater_is_better=True,
      learning_rate=learning_rate,
      #max_grad_norm = 1.0,
      run_name=nome
    )

    trainer = SentenceTransformerTrainer(
      model=model,
      args=args,
      train_dataset=dataset_treino,
      eval_dataset=dataset_val,
      loss=train_loss,
      evaluator=RecallAtKEvaluator()
    )

    trainer.train()

    run.finish()

    model.save(folder+"/modelos/"+nome)

    model.eval()
    recall_1_teste = recall_at_k(model,1,corpus_teste,queries_teste)
    recall_3_teste = recall_at_k(model,3,corpus_teste,queries_teste)
    recall_7_teste = recall_at_k(model,7,corpus_teste,queries_teste)
    print(f"Recall@1: {recall_1_teste}")
    print(f"Recall@3: {recall_3_teste}")
    print(f"Recall@7: {recall_7_teste}")

    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@1'] = recall_1_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@3'] = recall_3_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@7'] = recall_7_teste

    resultados.to_csv(folder+"/resultados_finetuning.csv")

Treinando modelo 'all-MiniLM-L6-v2' (modelo 1 de 6) com learning rate de 5e-06:


wandb: Currently logged in as: ana-sovat (ana-sovat-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Recall@1,Recall@3,Recall@7
100,0.055500,0.036003,0.944099,0.990683,0.998447
200,0.041000,0.031436,0.933230,0.986025,0.996894
300,0.037000,0.029425,0.930124,0.986025,0.996894
400,0.033600,0.027935,0.923913,0.984472,0.998447
500,0.034300,0.026734,0.923913,0.984472,0.998447
600,0.032500,0.026330,0.923913,0.990683,0.998447
700,0.030600,0.025163,0.920807,0.989130,0.995342
800,0.027900,0.024539,0.927019,0.990683,0.996894
900,0.030600,0.024075,0.925466,0.989130,0.998447
1000,0.029600,0.023448,0.928571,0.990683,0.998447


## Continuação depois de interrompido

In [ ]:
modelos = [
    "multi-qa-distilbert-cos-v1",
    "multi-qa-MiniLM-L6-cos-v1",
]

learning_rates = [5e-06,5e-05]

In [ ]:
resultados = pd.read_csv(folder+"/resultados_finetuning.csv")
resultados

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
0,0,0,0,all-MiniLM-L6-v2,0.000005,0.924383,0.976852,0.996914
1,1,1,1,all-MiniLM-L12-v2,0.000005,0.955247,0.993827,1.000000
2,2,2,2,multi-qa-mpnet-base-dot-v1,0.000005,0.958333,0.993827,1.000000
3,3,3,3,multi-qa-mpnet-base-cos-v1,0.000005,0.969136,0.996914,1.000000
4,4,4,4,multi-qa-distilbert-cos-v1,0.000005,0.942901,0.989198,0.996914
5,5,5,5,multi-qa-MiniLM-L6-cos-v1,0.000005,0.929012,0.983025,0.993827
6,6,6,6,all-MiniLM-L6-v2,0.000050,0.950617,0.983025,0.998457
7,7,7,7,all-MiniLM-L12-v2,0.000050,0.967593,0.995370,1.000000
8,8,8,8,multi-qa-mpnet-base-dot-v1,0.000050,0.961420,0.989198,0.998457
9,9,9,9,multi-qa-mpnet-base-cos-v1,0.000050,0.958333,0.995370,0.998457


In [ ]:
for i,modelo in enumerate(modelos):
  for learning_rate in learning_rates:
    print(f"Treinando modelo '{modelo}' (modelo {i+1} de 2) com learning rate de {learning_rate}:")
    nome = f"{modelo}_{learning_rate}"
    epocas = 4

    run = wandb.init(project="conserta-avioes_fine-tuning_01",name=nome)

    model = SentenceTransformer(modelo)
    train_loss = losses.CosineSimilarityLoss(model=model)

    args = SentenceTransformerTrainingArguments(
      output_dir=nome,
      num_train_epochs=epocas,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      eval_strategy="steps",
      eval_steps=100,
      save_strategy="steps",
      save_steps=100,
      save_total_limit=2,
      logging_steps=100,
      load_best_model_at_end=True,
      metric_for_best_model="recall@1",
      greater_is_better=True,
      learning_rate=learning_rate,
      #max_grad_norm = 1.0,
      run_name=nome
    )

    trainer = SentenceTransformerTrainer(
      model=model,
      args=args,
      train_dataset=dataset_treino,
      eval_dataset=dataset_val,
      loss=train_loss,
      evaluator=RecallAtKEvaluator()
    )

    trainer.train()

    run.finish()

    model.save(folder+"/modelos/"+nome)

    model.eval()
    recall_1_teste = recall_at_k(model,1,corpus_teste,queries_teste)
    recall_3_teste = recall_at_k(model,3,corpus_teste,queries_teste)
    recall_7_teste = recall_at_k(model,7,corpus_teste,queries_teste)
    print(f"Recall@1: {recall_1_teste}")
    print(f"Recall@3: {recall_3_teste}")
    print(f"Recall@7: {recall_7_teste}")

    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@1'] = recall_1_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@3'] = recall_3_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@7'] = recall_7_teste

    resultados.to_csv(folder+"/resultados_finetuning.csv")

Treinando modelo 'multi-qa-distilbert-cos-v1' (modelo 1 de 2) com learning rate de 5e-06:


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: ana-sovat (ana-sovat-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Recall@1: 0.9290123456790124
Recall@3: 0.9830246913580247
Recall@7: 0.9938271604938271
Treinando modelo 'multi-qa-MiniLM-L6-cos-v1' (modelo 2 de 2) com learning rate de 5e-05:


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Recall@1,Recall@3,Recall@7
100,0.047500,0.030605,0.940994,0.981366,0.995342
200,0.033300,0.026926,0.925466,0.975155,0.989130
300,0.033500,0.027619,0.922360,0.973602,0.982919
400,0.032800,0.025698,0.920807,0.964286,0.979814
500,0.027400,0.025383,0.936335,0.968944,0.987578
600,0.027600,0.023587,0.933230,0.973602,0.989130
700,0.022300,0.021665,0.939441,0.976708,0.984472
800,0.021400,0.021765,0.920807,0.968944,0.989130
900,0.019000,0.020669,0.933230,0.975155,0.986025
1000,0.020700,0.020093,0.934783,0.979814,0.989130


eval/loss,█▆▆▆▅▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/recall@1,▂▁▁▄▁▅▄▆▅▆▆▆▆▃▇█▆▇▄▃▅▅▆▆▅▅▅▄▄▅▄▅▄▄▄▅▄▅▅▅
eval/recall@3,▇▄▁▃▅▆▂▅▇▅▃█▅▄▅▅█▅▃▃▅▅▅█▅▅▅▅▆▅▅▅▅▅▆▅▅▄▅▅
eval/recall@7,▅▁▃▅▃▆█▅▃▆▄▄▅▄▆▄▆▆▆▅▅▇▅▅▅▆▆▅▆▄▄▅▆▆▇▆▆▆▆▅
eval/runtime,▄▄▁▅▃▆▅▂▅▂▅▁▅▁▄▆▁▃▁▆▁▄▄▁█▂█▂▁▂▇▄▇▄▁▁▆▃▆▁
eval/samples_per_second,▄▄█▃▅▂▄▇▄▇▇▄█▃█▄▇▃▅█▂▆█▅▅▁█▇▁▇▇▂▆▇▂██▃██
eval/steps_per_second,▄▄█▃█▄▃▇▃▃▄█▄▃▄▅█▃▅██▃▅▅▇█▇▁▇▇▇▆▇▂██▃▅▂█
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,▆▃▆▇▃▂▄▃█▃▂▃▃▅▂▂▃▅▁▂▁▃▁▃▃▃▄▂▄▂▂▂▃▂▂▃▄▂▂▂
+2,...


Recall@1: 0.9429012345679012
Recall@3: 0.9861111111111112
Recall@7: 0.9969135802469136


In [ ]:
modelos = [
    "all-MiniLM-L6-v2",
    "all-MiniLM-L12-v2",
    "multi-qa-mpnet-base-cos-v1",
    "multi-qa-distilbert-cos-v1",
]

learning_rates = [5e-04]

In [ ]:
for i,modelo in enumerate(modelos):
  for learning_rate in learning_rates:
    print(f"Treinando modelo '{modelo}' (modelo {i+1} de 4) com learning rate de {learning_rate}:")
    nome = f"{modelo}_{learning_rate}"
    epocas = 4

    run = wandb.init(project="conserta-avioes_fine-tuning_01",name=nome)

    model = SentenceTransformer(modelo)
    train_loss = losses.CosineSimilarityLoss(model=model)

    args = SentenceTransformerTrainingArguments(
      output_dir=nome,
      num_train_epochs=epocas,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      eval_strategy="steps",
      eval_steps=100,
      save_strategy="steps",
      save_steps=100,
      save_total_limit=2,
      logging_steps=100,
      load_best_model_at_end=True,
      metric_for_best_model="recall@1",
      greater_is_better=True,
      learning_rate=learning_rate,
      #max_grad_norm = 1.0,
      run_name=nome
    )

    trainer = SentenceTransformerTrainer(
      model=model,
      args=args,
      train_dataset=dataset_treino,
      eval_dataset=dataset_val,
      loss=train_loss,
      evaluator=RecallAtKEvaluator()
    )

    trainer.train()

    run.finish()

    model.save(folder+"/modelos/"+nome)

    model.eval()
    recall_1_teste = recall_at_k(model,1,corpus_teste,queries_teste)
    recall_3_teste = recall_at_k(model,3,corpus_teste,queries_teste)
    recall_7_teste = recall_at_k(model,7,corpus_teste,queries_teste)
    print(f"Recall@1: {recall_1_teste}")
    print(f"Recall@3: {recall_3_teste}")
    print(f"Recall@7: {recall_7_teste}")

    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@1'] = recall_1_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@3'] = recall_3_teste
    resultados.loc[(resultados['Modelo Base']==modelo)&(resultados['Learning Rate']==learning_rate),'Recall@7'] = recall_7_teste

    resultados.to_csv(folder+"/resultados_finetuning.csv")

Treinando modelo 'all-MiniLM-L6-v2' (modelo 1 de 4) com learning rate de 0.0005:


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Recall@1,Recall@3,Recall@7
100,0.119600,0.129504,0.397516,0.551242,0.667702
200,0.340000,0.337831,0.062112,0.136646,0.250000
300,0.425700,0.500000,0.009317,0.029503,0.057453
400,0.483300,0.500000,0.006211,0.017081,0.040373
500,0.337700,0.500000,0.001553,0.013975,0.035714
600,0.273700,0.500000,0.006211,0.017081,0.060559
700,0.261600,0.500000,0.003106,0.012422,0.035714
800,0.258300,0.500000,0.000000,0.009317,0.038820
900,0.262900,0.500000,0.010870,0.027950,0.052795
1000,0.264400,0.500000,0.004658,0.017081,0.040373


eval/loss,▁███████████████████████████████████████
eval/recall@1,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/recall@3,█▂▂▁▂▁▂▁▁▂▁▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂
eval/recall@7,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▅▃▁▅▁▆▅▃▄▃▅▃▂▂▂▇▂▃█▂▄▃▂▅▃▄▆▄▂█▂▇▆▇▅▅▃▄█▅
eval/samples_per_second,█▆▇█▄▃▃▄▆▆▆▆▆▇▇▂▄▆▆▃▂▅▄▆▇▁▆▅▃▃▄▁▇▃▃▆▅▁▃▇
eval/steps_per_second,▄█▆▇█▄▂▃▂▄▆▅▆▇▇▂▄▆▅▇▅▃▆▇▄▆▇▃▅▃▆▆▂▃▂▄▆▅▁▇
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇███
train/grad_norm,█▆▁▃▁▂▂▂▂▁▂▂▁▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▂▂▂▁▂▁▁▂▁▁▁▃
+2,...


Recall@1: 0.42746913580246915
Recall@3: 0.5864197530864198
Recall@7: 0.7037037037037037


In [ ]:
novos = pd.DataFrame({"Modelo Base":modelos,"Learning Rate":[5e-04]*4,"Recall@1":[None]*4,"Recall@3":[None]*4,"Recall@7":[None]*4})
novos

,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
0,all-MiniLM-L6-v2,0.0005,None,None,None
1,all-MiniLM-L12-v2,0.0005,None,None,None
2,multi-qa-mpnet-base-cos-v1,0.0005,None,None,None
3,multi-qa-distilbert-cos-v1,0.0005,None,None,None


In [ ]:
for i,modelo in enumerate(modelos):
  for learning_rate in learning_rates:
    nome = f"{modelo}_{learning_rate}"
    model = SentenceTransformer(folder+"/modelos/"+nome)

    model.eval()
    recall_1_teste = recall_at_k(model,1,corpus_teste,queries_teste)
    recall_3_teste = recall_at_k(model,3,corpus_teste,queries_teste)
    recall_7_teste = recall_at_k(model,7,corpus_teste,queries_teste)
    print(f"Recall@1: {recall_1_teste}")
    print(f"Recall@3: {recall_3_teste}")
    print(f"Recall@7: {recall_7_teste}")

    novos.loc[(novos['Modelo Base']==modelo)&(novos['Learning Rate']==learning_rate),'Recall@1'] = recall_1_teste
    novos.loc[(novos['Modelo Base']==modelo)&(novos['Learning Rate']==learning_rate),'Recall@3'] = recall_3_teste
    novos.loc[(novos['Modelo Base']==modelo)&(novos['Learning Rate']==learning_rate),'Recall@7'] = recall_7_teste

Recall@1: 0.8425925925925926
Recall@3: 0.9367283950617284
Recall@7: 0.9861111111111112
Recall@1: 0.6635802469135802
Recall@3: 0.7978395061728395
Recall@7: 0.8734567901234568
Recall@1: 0.004629629629629629
Recall@3: 0.006172839506172839
Recall@7: 0.026234567901234566
Recall@1: 0.42746913580246915
Recall@3: 0.5864197530864198
Recall@7: 0.7037037037037037


In [ ]:
    resultados2 = pd.concat([resultados,novos])
    resultados2

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
0,0.0,0.0,0.0,all-MiniLM-L6-v2,0.000005,0.924383,0.976852,0.996914
1,1.0,1.0,1.0,all-MiniLM-L12-v2,0.000005,0.955247,0.993827,1.0
2,2.0,2.0,2.0,multi-qa-mpnet-base-dot-v1,0.000005,0.958333,0.993827,1.0
3,3.0,3.0,3.0,multi-qa-mpnet-base-cos-v1,0.000005,0.969136,0.996914,1.0
4,4.0,4.0,4.0,multi-qa-distilbert-cos-v1,0.000005,0.942901,0.989198,0.996914
5,5.0,5.0,5.0,multi-qa-MiniLM-L6-cos-v1,0.000005,0.929012,0.983025,0.993827
6,6.0,6.0,6.0,all-MiniLM-L6-v2,0.000050,0.950617,0.983025,0.998457
7,7.0,7.0,7.0,all-MiniLM-L12-v2,0.000050,0.967593,0.99537,1.0
8,8.0,8.0,8.0,multi-qa-mpnet-base-dot-v1,0.000050,0.96142,0.989198,0.998457
9,9.0,9.0,9.0,multi-qa-mpnet-base-cos-v1,0.000050,0.958333,0.99537,0.998457


In [ ]:
resultados2 = resultados2.iloc[:,3:]
resultados2

,Modelo Base,Learning Rate,Recall@1,Recall@3,Recall@7
0,all-MiniLM-L6-v2,0.000005,0.924383,0.976852,0.996914
1,all-MiniLM-L12-v2,0.000005,0.955247,0.993827,1.0
2,multi-qa-mpnet-base-dot-v1,0.000005,0.958333,0.993827,1.0
3,multi-qa-mpnet-base-cos-v1,0.000005,0.969136,0.996914,1.0
4,multi-qa-distilbert-cos-v1,0.000005,0.942901,0.989198,0.996914
5,multi-qa-MiniLM-L6-cos-v1,0.000005,0.929012,0.983025,0.993827
6,all-MiniLM-L6-v2,0.000050,0.950617,0.983025,0.998457
7,all-MiniLM-L12-v2,0.000050,0.967593,0.99537,1.0
8,multi-qa-mpnet-base-dot-v1,0.000050,0.96142,0.989198,0.998457
9,multi-qa-mpnet-base-cos-v1,0.000050,0.958333,0.99537,0.998457


In [ ]:
resultados2.to_csv(folder+"/resultados_finetuning.csv",index=False)